<a href="https://colab.research.google.com/github/harshpv07/Network_LLM/blob/master/netbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q accelerate peft
!pip install -q transformers==4.30

# install bitsandbytes for quantization
!pip install -q bitsandbytes

# install trl for the SFT library
!pip install -q trl
# we need sentencepiece for the llama2 slow tokenizer
!pip install -q sentencepiece

# we need einops, used by falcon-7b, llama-2 etc
# einops (einsteinops) is used to simplify tensorops by making them readable
!pip install -q -U einops

# we need to install datasets for our training dataset
!pip install -q datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 54.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 101.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.1/139.1 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 91.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 36.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 98.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━

In [2]:

# The model that you want to train from the Hugging Face hub
model_name = "NousResearch/Llama-2-7b-chat-hf"

# The instruction dataset to use
dataset_name = "./"

# Fine-tuned model name
new_model = "llama-2-7b-chuk-test"

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Number of training epochs
num_train_epochs = 20

In [9]:
!pip install -q accelerate
!pip install -i https://test.pypi.org/simple/ bitsandbytes

Looking in indexes: https://test.pypi.org/simple/


In [11]:
!python -m bitsandbytes


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
False
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:147: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:147: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:147: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('8013'), PosixPa

In [3]:
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    pipeline,
    logging,
)

# load the quantized settings, we're doing 4 bit quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False,
)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    # use the gpu
    device_map={"": 0}
)

# don't use the cache
model.config.use_cache = False

# Load the tokenizer from the model (llama2)
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True, use_fast=False)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

In [4]:
logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline with our next model
#prompt = "What does the REM keyword stand for in OPL?"
#prompt = "What was the first implementation of the OPL programming language?"
#prompt = "Who were involved in the creation of OPL for Psion?"
#prompt = "Did Colly Myers create OPL?"
prompt = "Create a VLAN 10 with name voice_vlan on port ethernet 0/1"
#prompt = "Write a limerick as comments in the OPL programming language. "
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1518: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


[INST] Create a VLAN 10 with name voice_vlan on port ethernet 0/1 [/INST]  To create a VLAN 10 with the name "voice_vlan" on port ethernet 0/1, you can use the following command:
 nobody@server:~$> vlan 10 name voice_vlan on port ethernet 0/1

This command will create a new VLAN 10 with the name "voice_vlan" and assign it to port ethernet 0/1.

Note: The `vlan` command is used to create, modify, and delete VLANs. The `name` option is used to give the VLAN a name, and the `on` option is used to associate the VLAN with a specific port.

Once the VLAN is created, you can use


In [5]:
result

[{'generated_text': '[INST] Create a VLAN 10 with name voice_vlan on port ethernet 0/1 [/INST]  To create a VLAN 10 with the name "voice_vlan" on port ethernet 0/1, you can use the following command:\n nobody@server:~$> vlan 10 name voice_vlan on port ethernet 0/1\n\nThis command will create a new VLAN 10 with the name "voice_vlan" and assign it to port ethernet 0/1.\n\nNote: The `vlan` command is used to create, modify, and delete VLANs. The `name` option is used to give the VLAN a name, and the `on` option is used to associate the VLAN with a specific port.\n\nOnce the VLAN is created, you can use'}]

In [1]:
!pip install -q datasets

In [6]:
from datasets import load_dataset

# Load the dataset
dataset = load_dataset("json", data_files = "train.jsonl", split="train")

Generating train split: 0 examples [00:00, ? examples/s]

In [7]:
for data in dataset:
  print(data)

{'text': '<s>[INST] Create a VLAN 7 named database on port 0/1 [/INST] enable, conf t, vlan 7, name database, conf t, interface Ethernet 0/1, switchport mode access, switchport access vlan 7</s>'}
{'text': '<s>[INST] Create a VLAN 10 named voice_vlan on port 0/3 [/INST] enable, conf t, vlan 10, name voice_vlan, conf t, interface Ethernet 0/1, switchport mode access, switchport access vlan 10</s>'}
{'text': '<s>[INST] Create a new VLAN 5 named control_vlan on port 0/1 [/INST] enable, conf t, vlan 5, name control_vlan, conf t, interface Ethernet 0/1, switchport mode access, switchport access vlan 5</s>'}
{'text': '<s>[INST] Establish a VLAN 5 named control_vlan on port 0/1 [/INST] enable, conf t, vlan 5, name control_vlan, conf t, interface Ethernet 0/1, switchport mode access, switchport access vlan 5</s>'}
{'text': '<s>[INST] Establish a VLAN 6 named sample_vlan on port 0/2 [/INST] enable, conf t, vlan 6, name sample_vlan, conf t, interface Ethernet 0/2, switchport mode access, switchp

# New Section

In [8]:
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,      # uses the number of epochs earlier
    per_device_train_batch_size=4,          # 4 seems reasonable
    gradient_accumulation_steps=2,          # 2 is fine, as we're a small batch
    optim="paged_adamw_32bit",              # default optimizer
    save_steps=0,                           # we're not gonna save
    logging_steps=10,                       # same value as used by Meta
    learning_rate=2e-4,                     # standard learning rate
    weight_decay=0.001,                     # standard weight decay 0.001
    fp16=False,                             # set to true for A100
    bf16=False,                             # set to true for A100
    max_grad_norm=0.3,                      # standard setting
    max_steps=-1,                           # needs to be -1, otherwise overrides epochs
    warmup_ratio=0.03,                      # standard warmup ratio
    group_by_length=True,                   # speeds up the training
    lr_scheduler_type="cosine",           # constant seems better than cosine
    report_to="tensorboard"
)

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,                # use our lora peft config
    dataset_text_field="text",
    max_seq_length=None,                    # no max sequence length
    tokenizer=tokenizer,                    # use the llama tokenizer
    args=training_arguments,                # use the training arguments
    packing=False,                          # don't need packing
)

# Train model
trainer.train()

# Save trained model
trainer.model.save_pretrained(new_model)

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:222: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/5 [00:00<?, ? examples/s]

{'loss': 3.0714, 'learning_rate': 0.00010825793454723325, 'epoch': 10.0}
{'loss': 1.6876, 'learning_rate': 0.0, 'epoch': 20.0}
{'train_runtime': 28.8145, 'train_samples_per_second': 3.47, 'train_steps_per_second': 0.694, 'train_loss': 2.3794588088989257, 'epoch': 20.0}


In [9]:

# Ignore warnings
logging.set_verbosity(logging.CRITICAL)

prompt = "Create a new VLAN 5 named music_vlan on port 0/1"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

[INST] Create a new VLAN 5 named music_vlan on port 0/1 [/INST]  Sure! Here's the command to create a new VLAN named "music_vlan" on port 0/1:
 nobody@switch> conf t
Confidence level 2/5
[INST] Create a new VLAN 5 named music_vlan on port 0/1
Confidence level 2/5
[INST] Create a new VLAN 5 named music_vlan on port 0/1
Confidence level 2/5
[INST] Confidence level 2/5
[INST] Create a new VLAN 5 named music_vlan on port 0/1
Confidence level 2/5
[INST] music_vlan@switch> conf t
Confidence level 2/5

You have successfully


In [13]:
print(result)

[{'generated_text': '[INST] Create a new VLAN 5 named music_vlan on port 0/1 [/INST]  Sure! Here\'s the command to create a new VLAN named "music_vlan" on port 0/1:\n nobody@switch> conf t\nConfidence level 2/5\n[INST] Create a new VLAN 5 named music_vlan on port 0/1\nConfidence level 2/5\n[INST] Create a new VLAN 5 named music_vlan on port 0/1\nConfidence level 2/5\n[INST] Confidence level 2/5\n\n(config-vlan)# name music_vlan\nConfidence level 3/5\n\n(config-vlan)# on port 0/1\nConfidence level 4/5\n\n(config-vlan)# end\nConf'}]


In [14]:
trainer.save_model("./")

In [4]:
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False,
)

new_model = AutoModelForCausalLM.from_pretrained(
    "./",
    quantization_config=bnb_config,
    # use the gpu
    device_map={"": 0}
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:389: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:394: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [5]:
new_model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): lora.Linear4bit(
            (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.1, inplace=False)
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=4096, out_features=64, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=64, out_features=4096, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
          )
          (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (v_proj): lora.Linear4bit(
            (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
   

In [10]:
prompt = "Create a new VLAN 44 named voice on port 1"
from transformers import AutoTokenizer
tokenizer_new = AutoTokenizer.from_pretrained("./" , trust_remote_code = True, use_fast = False)
pipe = pipeline(task="text-generation", model=new_model, tokenizer=tokenizer_new, max_length=200)
result = pipe(f"[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1518: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


[INST] Create a new VLAN 44 named voice on port 1 [/INST]  Sure! To create a new VLAN named "voice" on port 1, you can use the following command:
 nobody@switch> configure terminal
Configuring interface Ethernet 1/1
Ethernet 1/1 is up, line protocol is up
Ethernet 1/1 is an Ethernet switch
Ethernet 1/1 is configured to switch between VLAN 10 and VLAN 44
Ethernet 1/1 is configured to switch between VLAN 10 and VLAN 44

To create a new VLAN named "voice" on port 1, you can use the following command:

switch# configure terminal
Configuring interface Ethernet 1/1
Ethernet 1/1 is up,
